In [5]:
from pymongo import MongoClient
import polars as pl

In [6]:
# Conectar ao MongoDB
client = MongoClient("mongodb://localhost:27017")  # Conectar ao MongoDB local
db = client["ctt2025"]  # Nome da base de dados no MongoDB
collection = db["Expresso_2025"]  # Nome da coleção (tabela)

In [7]:


# Consulta de agregação
pipeline = [
    # Filtro para garantir que estamos lidando com dados válidos
    {"$match": {"DATA_CRIACAO": {"$exists": True}, "CENTRO": {"$exists": True}}},
    
    # Agrupar por DATA_CRIACAO e CENTRO, contar os documentos
    {"$group": {
        "_id": {"DATA_CRIACAO": "$DATA_CRIACAO", "CENTRO": "$CENTRO"},  # Agrupar por DATA e CENTRO
        "numero_de_objetos": {"$sum": 1}  # Contar o número de documentos em cada grupo
    }},
    
    # Ordenar pela DATA_CRIACAO
    {"$sort": {"_id.DATA_CRIACAO": 1}}  # Ordenar pela data
]

# Executar a agregação
resultado = collection.aggregate(pipeline)

# Coletar os resultados e transformar em uma lista de dicionários
dados = []
for item in resultado:
    dados.append({
        "centro": item["_id"]["CENTRO"],
        "data": item["_id"]["DATA_CRIACAO"],
        "numero_de_objetos": item["numero_de_objetos"]
    })

# Definir um schema explícito para o DataFrame
schema = [
    ("centro", pl.Utf8),
    ("data", pl.Utf8),
    ("numero_de_objetos", pl.Int64)
]

# Criar um DataFrame Polars com os dados e schema
df = pl.DataFrame(dados, schema=schema)

# Mostrar o DataFrame
print(df)


shape: (633, 3)
┌─────────┬────────────┬───────────────────┐
│ centro  ┆ data       ┆ numero_de_objetos │
│ ---     ┆ ---        ┆ ---               │
│ str     ┆ str        ┆ i64               │
╞═════════╪════════════╪═══════════════════╡
│ 8811417 ┆ 01/04/2025 ┆ 9515              │
│ 8811373 ┆ 01/04/2025 ┆ 16724             │
│ 8819009 ┆ 01/04/2025 ┆ 19296             │
│ 8817018 ┆ 01/04/2025 ┆ 19078             │
│ 8818154 ┆ 01/04/2025 ┆ 14765             │
│ …       ┆ …          ┆ …                 │
│ 8819001 ┆ 31/03/2025 ┆ 10426             │
│ 8811417 ┆ 31/03/2025 ┆ 9288              │
│ 8811496 ┆ 31/03/2025 ┆ 25844             │
│ 8811455 ┆ 31/03/2025 ┆ 14397             │
│ 8811456 ┆ 31/03/2025 ┆ 10034             │
└─────────┴────────────┴───────────────────┘


In [8]:
# Recuperar todos os dados da coleção
dados = []
for item in collection.find():
    dados.append({
        "centro": item.get("CENTRO"),
        "data": item.get("DATA_CRIACAO"),
        "giro": item.get("Giro"),
        "loptica": item.get("LOPTICA"),
        "janela_horaria": item.get("JANELA_HORARIA"),
        "nome": item.get("NOME"),
        "morada": item.get("MORADA"),
        "cp": item.get("CP"),
        "localidade": item.get("LOCALIDADE"),
        "cod_t_even": item.get("COD_T_EVEN"),
        "data_evento": item.get("DATA_EVENTO"),
        "latitude": item.get("LATITUDE"),
        "longitude": item.get("LONGITUDE"),
        "nome_rem": item.get("NOME_REM"),
        "cod_pais_origem": item.get("COD_PAIS_ORIGEM")
    })

# Criar o DataFrame Polars com os dados
df = pl.DataFrame(dados)


In [9]:
df.head()

centro,data,giro,loptica,janela_horaria,nome,morada,cp,localidade,cod_t_even,data_evento,latitude,longitude,nome_rem,cod_pais_origem
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""Lisboa""","""2025-04-24""","""G123""","""LO1""","""08:00-10:00""","""João Silva""","""Rua das Flores, 123""","""1000-000""","""Lisboa""","""EVT456""","""2025-04-23""","""38.7169""","""-9.1399""","""Maria Silva""","""PT"""
"""8811373""","""25/03/2025""",null,"""DS726222159PT""","""09-19""","""FILIPA ALEXANDRA DE SOUSA BART…","""RUA DOUTOR JOSÉ JOAQUIM DE ALM…","""2775-595""","""CARCAVELOS""","""H""","""25/03/2025 21:11:05""","""38.82863972""","""-9.30678108""","""YUCAN - FULFILLMENT SERVICE FO…","""ES"""
"""8811373""","""25/03/2025""",null,"""DS725770471PT""","""09-19""","""VALERIO ANTUNES""","""RUA BARÃO DE MO AMEDES 79 2 FR…","""2775-715""","""CARCAVELOS""","""H""","""25/03/2025 21:10:50""","""38.82863972""","""-9.30678108""","""YUCAN - FULFILLMENT SERVICE FO…","""ES"""
"""8811373""","""25/03/2025""",null,"""DD173541954PT""","""09-19""","""CLARA MADEIRA""","""RUA HELIODORO SALGADO 56 ""","""2775-583""","""CARCAVELOS""","""H""","""25/03/2025 21:10:40""","""38.82863972""","""-9.30678108""","""AMINHAFARMACIA.PT""","""PT"""
"""8811373""","""25/03/2025""",null,"""DW705253646PT""","""09-19""","""NUNO FREIRE""","""RUA DOUTOR MANUEL DE ARRIAGA 4…","""2775-602""","""CARCAVELOS""","""H""","""25/03/2025 21:10:34""","""38.82863972""","""-9.30678108""","""AMAZON ROAD TRANSPORT SPAIN, S…","""ES"""


In [10]:
import polars as pl


# Converter o campo data para formato de data (caso esteja como string no formato 'YYYY.MM.DD')
df = df.with_columns([
    pl.col("data").str.strptime(pl.Date, "%Y.%m.%d", strict=False).alias("dia"),
    pl.col("cp").cast(pl.Utf8).fill_null("").alias("cp"),
    pl.col("nome").fill_null("Desconhecido").alias("cliente"),
    pl.col("cod_pais_origem").fill_null("??").alias("pais_de_origem"),
    pl.col("centro").fill_null("??").alias("centro")
])

# Agregações
resultado = (
    df.group_by(["dia", "cliente", "pais_de_origem", "centro"])
    .agg([
        pl.count().alias("n_objetos_total"),
        pl.col("cp").filter(pl.col("cp").str.len_chars() == 8).count().alias("n_objetos_cp7")
    ])
    .sort(["dia", "cliente"])
)

resultado

/var/folders/8m/hh_3__ks6m12ytdrpdgv3zxc0000gn/T/ipykernel_6432/150678575.py:17: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  pl.count().alias("n_objetos_total"),


dia,cliente,pais_de_origem,centro,n_objetos_total,n_objetos_cp7
date,str,str,str,u32,u32
null,""" PASCALE CHARONDI RE …","""PT""","""8811496""",2,2
null,""" CONTROLAUTO, SA A/C INFO…","""PT""","""8819002""",2,2
null,""" CONTROLAUTO, SA A/C MANU…","""PT""","""8819002""",1,1
null,""" JUSTINA NOIAS""","""ES""","""8811455""",4,4
null,""" A TOGA DE JOSE CARLOS LOUR…","""PT""","""8818171""",6,6
…,…,…,…,…,…
2025-04-28,"""ZUSETTE RODRIGUES""","""ES""","""8811373""",1,1
2025-04-28,"""ZUSLOG-SERV.LOGIST.UNIP.,LDA""","""PT""","""8818171""",2,2
2025-04-28,"""ZUZANNA CZAPLA""","""ES""","""8811373""",1,1


In [12]:
# Guardar como CSV
resultado.write_parquet("/Users/rr/Downloads/resumo_objetos_por_cliente.parquet")
df.write_parquet("/Users/rr/Downloads/Expresso_2025.parquet")

In [ ]:
df.write_csv("/Users/rr/Downloads/Expresso_2025.csv")


AttributeError: 'DataFrame' object has no attribute 'to_csv'